In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
def add_edges(G,a,b,connected_node,edges):
    G.add_edge(a,b)
    if nx.is_directed_acyclic_graph(G):
        edges -= 1
        connected_node.append(a)
        connected_node.append(b)
        connected_node = list(set(connected_node))
    else:
        G.remove_edge(a,b)
    return G,connected_node,edges

def random_dag(nodes, edges):
    """Generate a random Directed Acyclic Graph (DAG) with a given number of nodes and edges."""
    G = nx.DiGraph()
    for i in range(nodes):
        G.add_node(i)
        
    connected_node = []
    while edges > 0:
        a = np.random.randint(0,nodes)
        b=a
        while b==a:
            b = np.random.randint(0,nodes-1)
        if a not in G.neighbors(b) and b not in G.neighbors(a):
            if a in connected_node and b in connected_node:
                if np.random.uniform(0, 1)>0.7:
                    G,connected_node,edges = add_edges(G,a,b,connected_node,edges)

            else:
                if np.random.uniform(0, 1)>0.3:
                    G,connected_node,edges = add_edges(G,a,b,connected_node,edges)
                
    return G

def remove_DAG_edges(G,num_edges):
    random_edges_index = np.random.choice(len(list(G.edges)), size=num_edges, replace=False)
    random_edges_index = sorted(random_edges_index,reverse=True)
#     print(random_edges_index)
    for i in random_edges_index:
        random_edge = list(G.edges)[i]
        G.remove_edge(random_edge[0],random_edge[1])
    return G

def add_DAG_edges(G,num_edges):
    edges = num_edges
    nodes = len(list(G.nodes))
    
    components = nx.connected_components(G.to_undirected())
    connected_node = list([pred for pred in components][0])
    
    while edges > 0:
        a = np.random.randint(0,nodes)
        b=a
        while b==a:
            b = np.random.randint(0,nodes-1)
        if a not in G.neighbors(b) and b not in G.neighbors(a):
            G,connected_node,edges = add_edges(G,a,b,connected_node,edges)
#             if a in connected_node and b in connected_node:
#                 if np.random.uniform(0, 1)>0.7:
#                     G,connected_node,edges = add_edges(G,a,b,connected_node,edges)

#             else:
#                 if np.random.uniform(0, 1)>0.3:
#                     G,connected_node,edges = add_edges(G,a,b,connected_node,edges)
    return G

In [3]:
def generate_parameters(G,ratio_dis):
    num_cate = 4
    p = len(G.nodes)
    # sigma = np.random.uniform(1,2,(p,)) 
    n_con = int(p*(1-ratio_dis))
    n_dis = p - n_con
    type_node = np.concatenate((np.repeat(0,n_con), np.repeat(1,n_dis)))
    type_node = np.random.permutation(type_node)

    E_mean_con = np.random.uniform(0, 1,p)
    E_var_con = np.random.uniform(0.5, 1,p) # gaussion noise variance for continuous variable

    # C to C
    theta_cc = np.random.uniform(0.5, 1.5,(p,p)) * np.random.choice([-1, 1], (p,p)) # linear coefficient for continuous variable

    # D to D
    theta_dd = np.ones((p,p,num_cate,num_cate))
    theta_dd *= -1/4
    for i in range(p):
        for j in range(p):
            theta_dd[i,j] *= abs(theta_cc[i,j])
            theta_dd[i,j][np.diag_indices(num_cate)] *= -4
            theta_dd[i,j] = np.random.permutation(theta_dd[i,j])

    # D to C
    theta_dc = [[np.random.permutation([-theta_cc[i,j],-0.5*theta_cc[i,j],0.5*theta_cc[i,j],theta_cc[i,j]]) for j in range(p)] for i in range(p)]
    theta_dc = np.array(theta_dc)
    
    return type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc

def simulate_data(G,n,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc):
    num_cate = 4
    p = len(G.nodes)
    X = np.zeros((n,p))

    roots = []
    child = []
    for i in list(G.nodes):
        if len(nx.ancestors(G, i)) ==0:
            roots.append(i)
        else: 
            child.append(i)

    def exp_norm(weight):
        return np.exp(weight) / np.sum(np.exp(weight))

    for r in roots:
        node_idx = np.argwhere(np.array(list(G.nodes))==r)[0][0]
        if type_node[node_idx] == 0: # continuous node 
            X[:,r] = np.random.normal(E_mean_con[r], E_var_con[r],size = n)
    #         print(r)
        else:  # discrete node
            weight = [1/num_cate]*num_cate # + E_var_dis[r,]
            dis_counts = np.random.multinomial(1, exp_norm(weight), size=n)
            X[:,r] = np.argmax(dis_counts,axis=1)

            
    while len(roots) < p: 
        node_list = np.array(list(G.nodes))
        dis_nodes = node_list[type_node==1]
        con_nodes = node_list[type_node==0]

        for c in child:
            node_idx = np.argwhere(node_list==c)[0][0]

            # continuous child node
            if len(np.setdiff1d(list(nx.ancestors(G, c)),roots))==0 and type_node[node_idx] == 0:
                parent_idx = [pred for pred in G.predecessors(c)]

                parent_idx_dis = np.intersect1d(dis_nodes,parent_idx) # discrete parent node
                parent_idx_con = np.intersect1d(parent_idx,con_nodes) # continuous parent node

                # continuous par + Gaussian noise
                X[:,node_idx] = X[:,parent_idx_con] @ theta_cc[parent_idx_con,node_idx] + np.random.normal(0,E_var_con[node_idx],n)
                # discrete par
                if len(parent_idx_dis)>0:
                    for dis_idx in parent_idx_dis:
                        X[:,node_idx] += theta_dc[dis_idx,node_idx].flatten()[X[:,dis_idx].flatten().astype(int)]
#                 print(parent_idx,node_idx)
                child.remove(c);roots.append(c)

            # discrete child node 
            if len(np.setdiff1d(list(nx.ancestors(G, c)),roots))==0 and type_node[node_idx] == 1:
                parent_idx = [pred for pred in G.predecessors(c)]

                parent_idx_dis = np.intersect1d(dis_nodes,parent_idx) # discrete parent node
                parent_idx_con = np.intersect1d(parent_idx,con_nodes) # continuous parent node


                Weight = np.random.uniform(0, 1,(n,num_cate))
                if len(parent_idx_con)>0:
                    for con_idx in parent_idx_con:
                        Weight = X[:,[con_idx]] @ theta_dc[[con_idx],node_idx,] + Weight
                if len(parent_idx_dis)>0:
                    for dis_idx in parent_idx_dis:
                        Weight = theta_dd[dis_idx,node_idx][X[:,dis_idx].astype(int)] + Weight
                X[:,node_idx] = np.argmax(Weight,axis=1)


#                 print(parent_idx,node_idx)
                child.remove(c);roots.append(c)
    return X

In [4]:
def array2DF(X):
    df_x = pd.DataFrame(X)
    df_x.columns =[''.join(['X',str(i)])  for i in range(X.shape[1])]
    return df_x

def write_graph(G,filename):
    with open(filename, 'w') as f:
        f.write('Graph Nodes:')
        my_list = list(G.nodes)
        my_list = [''.join(map(str, ['X',i])) for i in my_list]
        my_string = ';'.join(map(str, my_list))
        f.writelines('\n')
        f.write(my_string)
        f.writelines('\n')
        f.writelines('\n')
        f.write('Graph Edges:')
        edge_list = list(G.edges)
        for i in range(len(edge_list)):
            f.writelines('\n')
            f.write(''.join(map(str, [i+1,".", ' X', edge_list[i][0],' --> X', edge_list[i][1] ])))
        f.writelines('\n')

In [5]:
def write_parameter(G,filename,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc):
    with open(filename, 'w') as f:
            roots = []
            child = []

            f.write('Graph Nodes:')
            f.writelines('\n')
            node_list = list(G.nodes)
            str_list = [''.join(map(str,["X",i, ": "])) for i in node_list]

            for i in range(len(node_list)):
                if len(nx.ancestors(G, i)) ==0:
                    roots.append(node_list[i])
                    if (type_node[i] == 0):
                        str_list[i] = ''.join(map(str,["X",i, ": ",'N(', E_mean_con[i] ,',',E_var_con[i],")"]))
                    else: 
                        str_list[i] = ''.join(map(str,["X",i, ": ", 'Dis( U(0,1), 1, 1, 1, 1)']))
                else: 
                    child.append(node_list[i])

            while len(roots) < len(node_list): 
                node_list = np.array(list(G.nodes))
                dis_nodes = node_list[type_node==1]
                con_nodes = node_list[type_node==0]
                for c in child:
                    node_idx = np.argwhere(node_list==c)[0][0]
                    # continuous child node
                    if len(np.setdiff1d(list(nx.ancestors(G, c)),roots))==0 and type_node[node_idx] == 0:
                        parent_idx = [pred for pred in G.predecessors(c)]
                        parent_idx_dis = np.intersect1d(dis_nodes,parent_idx) # discrete parent node
                        parent_idx_con = np.intersect1d(parent_idx,con_nodes) # continuous parent node

                        # continuous par + Gaussian noise
                        if len(parent_idx_con)>0:
                            for con_idx in parent_idx_con:
                                str_list[c] = ''.join(map(str,[str_list[c],  'X', con_idx, ' @ ' , theta_cc[con_idx,node_idx] , ' + '  ]))

                        if len(parent_idx_dis)>0:
                            for dis_idx in parent_idx_dis:
                                str_list[c] = ''.join(map(str,[str_list[c],  'Switch(','X', dis_idx, theta_dc[dis_idx,node_idx].flatten() , ')' , ' + '  ]))
                        str_list[c] = ''.join(map(str,[str_list[c], 'N(0,',E_var_con[node_idx] ,')']))
                        child.remove(c);roots.append(c)

                    if len(np.setdiff1d(list(nx.ancestors(G, c)),roots))==0 and type_node[node_idx] == 1:
                        parent_idx = [pred for pred in G.predecessors(c)]
                        parent_idx_dis = np.intersect1d(dis_nodes,parent_idx) # discrete parent node
                        parent_idx_con = np.intersect1d(parent_idx,con_nodes) # continuous parent node
                        str_list[c] = ''.join(map(str,[str_list[c], 'Dis( U(0,1), ']))

                        dis_string  = ['','','','']
                        if len(parent_idx_con)>0:
                            for con_idx in parent_idx_con:
                                for T in range(4):
                                    dis_string[T] =  ''.join(map(str,[dis_string[T],  'X', con_idx, ' @ ' , theta_dc[[con_idx],node_idx,].flatten()[T] , ' + '  ]))

                        if len(parent_idx_dis)>0:
                            for dis_idx in parent_idx_dis:
                                for T in range(4):
                                    dis_string[T] =  ''.join(map(str,[dis_string[T],  'Switch(','X' ,dis_idx, theta_dd[dis_idx,node_idx][T],  ' + '  ]))

                        dis_string = [i[:(len(i)-2)] for i in dis_string]
                        dis_string = ', '.join(dis_string)
                        str_list[c] = ''.join(map(str,[str_list[c], dis_string,  ')']))
                        child.remove(c);roots.append(c)
            for i in range(len(str_list)):
                f.write(str_list[i])
                f.writelines('\n')

In [15]:
os.makedirs("./Simulation_test/Dataset_75_05",exist_ok=True)

In [19]:
for i in range(10):

    G = random_dag(75,75) # generate dag with number of nodes and edges
    n = 1000 # sample number
    ratio_dis = 0.33 # discrete variable ratio
    edge_precentage = 0.05 # precentage of edge deleted & added
    num_edges = int(len(list(G.edges))*edge_precentage+0.4)
    
    
    G_1 = G.copy()
    G_2 = G.copy()
    G_3 = G.copy()



    G_1 = remove_DAG_edges(G_1,num_edges)
    G_1 = add_DAG_edges(G_1,num_edges)

    G_2 = remove_DAG_edges(G_2,num_edges)
    G_2 = add_DAG_edges(G_2,num_edges)

    G_3 = remove_DAG_edges(G_3,num_edges)
    G_3 = add_DAG_edges(G_3,num_edges)



    type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc = generate_parameters(G,ratio_dis)
    X = simulate_data(G,n,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    df_x = array2DF(X)

    X1 = simulate_data(G_1,n,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    df_x1 = array2DF(X1)
    X2 = simulate_data(G_2,n,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    df_x2 = array2DF(X2)
    X3 = simulate_data(G_3,n,type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    df_x3 = array2DF(X3)
    
    os.makedirs(''.join(['./Data',str(i)]),exist_ok=True)
    os.makedirs(''.join(['./Data',str(i),'/data']),exist_ok=True)
    df_x.to_csv(''.join(['./Data',str(i),'/data/data0.txt']), header=True, index=False, sep='\t', mode='w')
    df_x1.to_csv(''.join(['./Data',str(i),'/data/data1.txt']), header=True, index=False, sep='\t', mode='w')
    df_x2.to_csv(''.join(['./Data',str(i),'/data/data2.txt']), header=True, index=False, sep='\t', mode='w')
    df_x3.to_csv(''.join(['./Data',str(i),'/data/data3.txt']), header=True, index=False, sep='\t', mode='w')

    os.makedirs(''.join(['./Data',str(i),'/graph']),exist_ok=True)
    
    write_graph(G,''.join(['./Data',str(i),'/graph/graph0.txt']))
    write_graph(G_1,''.join(['./Data',str(i),'/graph/graph1.txt']))
    write_graph(G_2,''.join(['./Data',str(i),'/graph/graph2.txt']))
    write_graph(G_3,''.join(['./Data',str(i),'/graph/graph3.txt']))
    
    write_parameter(G,''.join(['./Data',str(i),'/graph/parameter0.txt']),type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    write_parameter(G_1,''.join(['./Data',str(i),'/graph/parameter1.txt']),type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    write_parameter(G_2,''.join(['./Data',str(i),'/graph/parameter2.txt']),type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)
    write_parameter(G_3,''.join(['./Data',str(i),'/graph/parameter3.txt']),type_node,E_mean_con,E_var_con,theta_cc,theta_dd,theta_dc)

In [ ]:
options = {
    "font_size": 20,
    "node_size": 900,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 1,
    "width": 2,
    "with_labels":True,
}

fig = plt.figure(figsize=(24, 6))
subax1 = plt.subplot(141)
pos = nx.circular_layout(G) 
subax1.title.set_text('Reference Graph')
nx.draw(G,pos=pos, **options)
subax2 = plt.subplot(142)
nx.draw(G_1,pos=pos, **options)
subax2.title.set_text('Case 1')
subax3 = plt.subplot(143)
nx.draw(G_2,pos=pos, **options)
subax3.title.set_text('Case 2')
subax4 = plt.subplot(144)
nx.draw(G_3,pos=pos, **options)
subax4.title.set_text('Case 3')

In [ ]:
fig = plt.figure(figsize=(10, 10))
subax1 = plt.subplot(221)
pos = nx.circular_layout(G) 
subax1.title.set_text('Reference Graph')
nx.draw(G,pos=pos, **options)
subax2 = plt.subplot(222)
nx.draw(G_1,pos=pos, **options)
subax2.title.set_text('Case 1')
subax3 = plt.subplot(223)
nx.draw(G_2,pos=pos, **options)
subax3.title.set_text('Case 2')
subax4 = plt.subplot(224)
nx.draw(G_3,pos=pos, **options)
subax4.title.set_text('Case 3')

In [ ]:
import pandas as pd
import pydot
from pycausal import search as s

In [ ]:
from pycausal.pycausal import pycausal as pc
pc = pc()
pc.start_vm()

In [ ]:
from pycausal import search as s
tetrad = s.tetradrunner()
tetrad.listScores()

In [ ]:
tetrad.getAlgorithmParameters(algoId = 'fges', scoreId = 'degen-gauss-bic')
tetrad.run(algoId = 'fges', dfs = df_x, scoreId = 'degen-gauss-bic', 
           dataType = 'mixed', numCategoriesToDiscretize = 4, maxDegree = 4, faithfulnessAssumed = True, 
           symmetricFirstStep = True, verbose = False)
print(len(tetrad.getEdges()))
tetrad.getEdges()

In [ ]:
tetrad.run(algoId = 'fges', dfs = df_x1, scoreId = 'degen-gauss-bic', 
           dataType = 'mixed', numCategoriesToDiscretize = 4, maxDegree = 4, faithfulnessAssumed = True, 
           symmetricFirstStep = True, verbose = False)
print(len(tetrad.getEdges()))
tetrad.getEdges()

In [ ]:
pc.stop_vm()